In [1]:
import requests
from bs4 import BeautifulSoup as soup
from requests_html import AsyncHTMLSession
import pandas as pd
import json
import re

In [2]:
header = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 12_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'}


In [3]:
html = requests.get('https://www.aliexpress.com/wholesale?catId=0&initiative_id=SB_20220803130344&SearchText=microprocessor&spm=a2g0o.home.1000002.0',headers=header)


In [4]:
html.status_code


200

In [5]:
cookies = html.cookies

In [6]:
bsobj = soup(html.content, "html.parser")

In [7]:
next = bsobj.find(title='Next Page')
next

In [8]:
bsobj

<!DOCTYPE html>

<html lang="en" xmlns:th="http://www.w3.org/1999/xhtml">
<head>
<meta charset="utf-8"/>
<title>microprocessor - Buy microprocessor with free shipping on AliExpress</title>
<meta content="microprocessor" name="keywords">
<meta content="Quality microprocessor with free worldwide shipping on AliExpress" name="description"/>
<meta content="follow,index" name="robots"/>
<link href="https://www.aliexpress.com/w/wholesale-microprocessor.html" rel="canonical"/>
<link href="https://www.aliexpress.com/w/wholesale-microprocessor.html" hreflang="en" rel="alternate">
<link href="https://id.aliexpress.com/w/wholesale-microprocessor.html" hreflang="id" rel="alternate"/>
<link href="https://ko.aliexpress.com/w/wholesale-microprocessor.html" hreflang="ko" rel="alternate"/>
<link href="https://ar.aliexpress.com/w/wholesale-microprocessor.html" hreflang="ar" rel="alternate"/>
<link href="https://de.aliexpress.com/w/wholesale-microprocessor.html" hreflang="de" rel="alternate"/>
<link href

In [9]:
productlist = bsobj.findAll('script',type="text/javascript")
products = productlist[3]

li = json.loads(re.findall(r"window.runParams\s*=\s*({.+?});", str(products))[0])

links = []
for i in li["mods"]["itemList"]["content"]:
    links.append(i["productId"])
links

['3256801728504039',
 '2255800243315514',
 '3256803146302318',
 '2251832648789993',
 '3256803850203105',
 '3256802567556682',
 '3256803932178243',
 '3256803701068138',
 '2251832701321414',
 '3256804390523899',
 '3256804336157932',
 '3256803932003055',
 '3256803906086259',
 '2255800363220505',
 '3256804390759826',
 '3256803263737228',
 '2251832698042672',
 '3256803591005661',
 '3256803413504396',
 '3256803874280378',
 '3256804180825086',
 '2251832650992137',
 '3256802842359444',
 '3256801740902059',
 '3256803371028970',
 '3256803051289148',
 '3256803164782884',
 '3256803197625796',
 '2251832057017116',
 '3256804387360941',
 '3256801527377689',
 '3256802984722084',
 '3256803886015863',
 '3256804417876325',
 '3256804369328813',
 '3256803543155360',
 '3256801579491800',
 '2251832806847539',
 '2251832765565400',
 '3256803740331927',
 '3256801917164262',
 '3256804227800573',
 '3256802171096885',
 '2251832673671212',
 '3256804056502496',
 '2251832603759114',
 '2251832699740637',
 '32568030595

In [10]:
asession = AsyncHTMLSession()
r = await asession.get('https://www.aliexpress.com')
for f in links:
    print('https://www.aliexpress.com/item/' + f + ".html")

https://www.aliexpress.com/item/3256801728504039.html
https://www.aliexpress.com/item/2255800243315514.html
https://www.aliexpress.com/item/3256803146302318.html
https://www.aliexpress.com/item/2251832648789993.html
https://www.aliexpress.com/item/3256803850203105.html
https://www.aliexpress.com/item/3256802567556682.html
https://www.aliexpress.com/item/3256803932178243.html
https://www.aliexpress.com/item/3256803701068138.html
https://www.aliexpress.com/item/2251832701321414.html
https://www.aliexpress.com/item/3256804390523899.html
https://www.aliexpress.com/item/3256804336157932.html
https://www.aliexpress.com/item/3256803932003055.html
https://www.aliexpress.com/item/3256803906086259.html
https://www.aliexpress.com/item/2255800363220505.html
https://www.aliexpress.com/item/3256804390759826.html
https://www.aliexpress.com/item/3256803263737228.html
https://www.aliexpress.com/item/2251832698042672.html
https://www.aliexpress.com/item/3256803591005661.html
https://www.aliexpress.com/i

In [11]:
df = pd.DataFrame(columns=['Product Id','In Stock','Rating','Shipping Time','Shipping Location'])

In [12]:
for f in links:
    
    
    temp = {}
    r = await asession.get('https://www.aliexpress.com/item/' + f + ".html")
    print('https://www.aliexpress.com/item/' + f + ".html")
    await r.html.arender()
    h1 = r.html.find('h1')
    temp['Product Id'] = h1[0].text
    stock = r.html.find('.product-quantity-info')
    if('\n' in stock[0].text):
        temp['In Stock'] = stock[0].text.split("\n")[1]
    else:
        temp['In Stock'] = stock[0].text
    rating = r.html.find('.overview-rating-average')
    if(rating):
        temp['Rating'] = rating[0].text
    else:
        temp['Rating'] = "None"
    rating_count = r.html.find('.dynamic-shipping-line.dynamic-shipping-contentLayout')
    if("Estimated" in rating_count[0].text.strip()):
        temp['Shipping Time'] = rating_count[0].text.strip()
        temp['Shipping Location'] = rating_count[1].text.strip()
    else:
        temp['Shipping Time'] = rating_count[1].text.strip()
        temp['Shipping Location'] = rating_count[0].text.strip()
    print(temp)
    df = df.append(temp, ignore_index=True)


https://www.aliexpress.com/item/3256801728504039.html
{'Product Id': 'AD9833 Programmable Microprocessors Serial Interface Module Sine Square Wave DDS Signal Generator Module GY-9833', 'In Stock': '4673 Pieces available', 'Rating': '5.0', 'Shipping Time': 'Estimated delivery on Oct 10', 'Shipping Location': 'From China to United States via Cainiao Super Economy Global'}
https://www.aliexpress.com/item/2255800243315514.html
{'Product Id': '1pcs/lot MC6803P MC6803 MC6803CP DIP-40 Microcontroller/Microprocessor (MCU/MPU)', 'In Stock': '9997 Pieces available', 'Rating': 'None', 'Shipping Time': 'Estimated delivery on Oct 10', 'Shipping Location': 'From China to United States via Cainiao Super Economy Global'}
https://www.aliexpress.com/item/3256803146302318.html
{'Product Id': '1PCS New Original R6502AP R6502 6502AP DIP40 Microprocessor Chip', 'In Stock': '9728 Pieces available', 'Rating': '5.0', 'Shipping Time': 'Estimated delivery on Oct 10', 'Shipping Location': 'From China to United St

In [13]:
display(df)

,Product Id,In Stock,Rating,Shipping Time,Shipping Location
0,AD9833 Programmable Microprocessors Serial Int...,4673 Pieces available,5.0,Estimated delivery on Oct 10,From China to United States via Cainiao Super ...
1,1pcs/lot MC6803P MC6803 MC6803CP DIP-40 Microc...,9997 Pieces available,None,Estimated delivery on Oct 10,From China to United States via Cainiao Super ...
2,1PCS New Original R6502AP R6502 6502AP DIP40 M...,9728 Pieces available,5.0,Estimated delivery on Oct 10,From China to United States via Cainiao Super ...
3,AD9833 Programmable Microprocessors Serial Int...,10397 Pieces available,5.0,Estimated delivery on Oct 10,From China to United States via Cainiao Super ...
4,1PCS ATMEGA2560-16AU ATMEGA2560 16U-TW TQFP-10...,20000 Pieces available,None,Estimated delivery on Aug 31,From China to United States via AliExpress Sta...
5,50 PCS MAX811TEUS SOT-143 APAA MAX811T MAX811 ...,1988 lots available,5.0,Estimated delivery on Sep 07,From China to United States via AliExpress Sta...
6,1PCS CP80C88-2 80C88 Direct-Plug DIP40 16-bit ...,9984 Pieces available,None,Estimated delivery on Oct 10,From China to United States via Cainiao Super ...
7,SP 705 706 707 708 R S T EN-L/TR MCU Monitor c...,13986 Pieces available,None,Estimated delivery on Oct 10,From China to United States via Cainiao Super ...
8,MC68008P10 MC68008P MC68008 DIP48 16-Bit Micro...,1278 Pieces available,5.0,Estimated delivery on Oct 10,From China to United States via Cainiao Super ...
9,New original IAP15W4K61S4-30 I - LQFP44 1 t 80...,10000 Pieces available,None,Estimated delivery on Sep 09,From China to United States via AliExpress Sta...
